In [ ]:
import pandas as pd
import numpy as np
import datetime
from datetime import datetime, timedelta, date
from tqdm import tqdm_notebook as tqdm

from multiprocessing import Pool
from functools import partial
import Training_function as func

In [ ]:
data = pd.read_csv('D:\\庫存健診開發\\data\\Training\\Raw\\Cluster_7.csv', converters={'ts': str, 'StockNo': str, 'StockName': str})
data['ts'] = pd.to_datetime(data['ts'])
data

In [ ]:
df_list = [group[1] for group in data.groupby(data['StockNo'])]
train_list, test_list = [], []

train_list, val_list, test_list = [], [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for j, x in enumerate(tqdm(pool.imap_unordered(partial(func.TrainTestSplit, TrainDate=date(2017,1,1), ValDate=date(2017,9,1)), df_list), total=len(df_list)), 1):
            train_list.append(x[0])
            val_list.append(x[1])
            test_list.append(x[2])

In [ ]:
train_list[3]

In [ ]:
df_list = [train_list, val_list, test_list]

X_train, Y_train, Y_train_ATR, X_val, Y_val, Y_val_ATR, X_test, Y_test, Y_test_ATR = [], [], [], [], [], [], [], [], []

if __name__ == '__main__':
    with Pool(processes=12) as pool:
        for i, stockdata in enumerate(df_list):
            if i == 0:
                for j, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset_classification, lookback=20, forward=1), stockdata), total=len(stockdata)), 1):
                    for xx in x[0]:
                        X_train.append(xx.tolist())
                    Y_train.extend(x[1])
                    Y_train_ATR.extend(x[2])
            elif i == 1:
                for j, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset_classification, lookback=20, forward=1), stockdata), total=len(stockdata)), 1):
                    for xx in x[0]:
                        X_val.append(xx.tolist())
                    Y_val.extend(x[1])
                    Y_val_ATR.extend(x[2])
            else:
                for j, x in enumerate(tqdm(pool.imap_unordered(partial(func.create_dataset_classification, lookback=20, forward=1), stockdata), total=len(stockdata)), 1):
                    for xx in x[0]:
                        X_test.append(xx.tolist())
                    Y_test.extend(x[1])
                    Y_test_ATR.extend(x[2])

In [ ]:
(len(X_train) == len(Y_train) == len(Y_train_ATR)) and (len(X_val) == len(Y_val) == len(Y_val_ATR)) and (len(X_test) == len(Y_test) == len(Y_test_ATR))

In [ ]:
Y_train, Y_train_ATR, Y_val, Y_val_ATR, Y_test, Y_test_ATR = func.categorical_transform(Y_train),  func.categorical_transform(Y_train_ATR), func.categorical_transform(Y_val), func.categorical_transform(Y_val_ATR), func.categorical_transform(Y_test), func.categorical_transform(Y_test_ATR)

In [ ]:
save_list = [X_train, Y_train, Y_train_ATR, X_val, Y_val, Y_val_ATR, X_test, Y_test, Y_test_ATR]

In [ ]:
cluster = 7
import pickle
with open(f'D:\\庫存健診開發\\data\\Training\\processed\\Cluster_{cluster}_classification_minmax0_Daily', 'wb') as fp:
    pickle.dump(save_list, fp)

In [ ]:
Y_train_ATR.count('flat')/len(Y_train)